<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/ProcessLogicIntegrated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeqSim Process Logic & Alarm Integration Tutorial

This notebook demonstrates the complete NeqSim process logic and alarm framework through two comprehensive examples:

## Part 1: Process Logic Integration
- ESD (Emergency Shutdown) Logic with multi-step sequences
- HIPPS (High Integrity Pressure Protection System) Logic
- Startup Logic with permissive conditions
- Scenario-based testing with batch execution API

## Part 2: Alarm-Triggered Process Logic
- Multi-level alarm configuration (LOLO, LO, HI, HIHI)
- Alarm-triggered automatic control actions
- Alarm-triggered ESD sequences
- Integrated safety architecture (5 layers of protection)

## Safety Architecture Overview

| Layer | Type | Setpoint | Action |
|-------|------|----------|--------|
| 1 | HI Alarm | 53 bara | Operator notification |
| 2 | HIHI Alarm | 56 bara | Automatic valve throttle |
| 3 | HIPPS (SIL-2) | 59 bara | Fast valve closure |
| 4 | ESD (SIL-2) | 60 bara | Emergency shutdown + blowdown |
| 5 | PSV (Mechanical) | 65 bara | Pressure relief |

## Import Required Libraries

First, import all necessary NeqSim classes for process equipment, logic, alarms, and safety scenarios.

In [ ]:
// Import NeqSim process equipment
import neqsim.process.equipment.separator.Separator;
import neqsim.process.equipment.stream.Stream;
import neqsim.process.equipment.splitter.Splitter;
import neqsim.process.equipment.valve.*;
import neqsim.process.equipment.mixer.Mixer;
import neqsim.process.equipment.flare.Flare;

// Import instrumentation
import neqsim.process.measurementdevice.*;

// Import process logic framework
import neqsim.process.logic.*;
import neqsim.process.logic.esd.ESDLogic;
import neqsim.process.logic.startup.StartupLogic;
import neqsim.process.logic.control.PressureControlLogic;
import neqsim.process.logic.action.*;
import neqsim.process.logic.condition.*;

// Import alarm system
import neqsim.process.alarm.*;

// Import scenario testing
import neqsim.process.safety.ProcessSafetyScenario;
import neqsim.process.util.scenario.*;

// Import core system
import neqsim.process.processmodel.ProcessSystem;
import neqsim.thermo.system.*;

System.out.println("✓ All libraries imported successfully");

---

# Part 1: Theory - Layers of Protection Analysis (LOPA)

## 1.1 Introduction to Process Safety

Process safety in oil & gas facilities relies on **multiple independent protection layers** to prevent hazardous events. This is formalized in the **Layers of Protection Analysis (LOPA)** methodology.

### Key Concepts:

**Independent Protection Layers (IPL):**
- Each layer must be independent of the others
- Failure of one layer should not affect other layers
- Layers are activated in sequence as risk increases

**Safety Integrity Level (SIL):**
- SIL-1: Risk reduction factor 10-100×
- SIL-2: Risk reduction factor 100-1000× (common for HIPPS, ESD)
- SIL-3: Risk reduction factor 1000-10000×

**Response Time Hierarchy:**
- Basic alarms: seconds to minutes (operator response)
- HIHI/LOLO alarms: automatic control (< 1 second)
- HIPPS: Fast-acting shutdown (2-5 seconds)
- ESD: Emergency isolation (5-30 seconds)
- PSV: Mechanical relief (instant, no logic required)

## 1.2 The Five Layers Implemented in This Example

| Layer | Type | Response | Function | SIL |
|-------|------|----------|----------|-----|
| **1** | HI/LO Alarm | Operator | Notification only | - |
| **2** | HIHI/LOLO Alarm | Automatic | Valve throttling | - |
| **3** | HIPPS | Automatic | Fast valve closure | 2 |
| **4** | ESD | Automatic | System isolation + blowdown | 2 |
| **5** | PSV | Mechanical | Pressure relief to flare | - |

### Layer Interaction:
```
Normal Operation (50 bara)
    ↓
HI Alarm (53 bara) → Operator warned
    ↓
HIHI Alarm (56 bara) → Auto throttle inlet valve
    ↓
HIPPS Activation (59 bara) → Fast close HIPPS valve (3 sec)
    ↓
ESD Activation (60 bara) → Close all inlets + open blowdown
    ↓
PSV Opens (65 bara) → Mechanical relief (last resort)
```

## 1.3 Theory: ESD (Emergency Shutdown) Logic

**Purpose:** Safely shut down the process when operating limits are exceeded.

**Key Characteristics:**
- **Time-sequenced actions**: Actions execute with specified delays
- **Fail-safe design**: Valves fail to safe position on power loss
- **Manual activation**: Operator can trigger via push button
- **Automatic triggers**: High pressure, temperature, level conditions

**ESD Sequence (Typical):**
1. **t=0s**: Close inlet valves (stop feed)
2. **t=0.5s**: Route gas to blowdown system
3. **t=0.5s**: Open blowdown valve (depressurize)
4. **t=1.0s**: Switch to transient calculation mode

**NeqSim Implementation:**
```java
ESDLogic esdLogic = new ESDLogic("ESD Level 1");
esdLogic.addAction(new CloseValveAction(inletValve), 0.0);      // Immediate
esdLogic.addAction(new CloseValveAction(esdInletValve), 0.0);   // Immediate
esdLogic.addAction(new SetSplitterAction(splitter, [0,0,1]), 0.5);  // Route to BD
esdLogic.addAction(new EnergizeESDValveAction(bdValve, 100.0), 0.5); // Open BD
```

## 1.4 Theory: HIPPS (High Integrity Pressure Protection System)

**Purpose:** Prevent overpressure by fast valve closure before PSV activation.

**Key Characteristics:**
- **Voting logic**: Typically 2oo3 (2 out of 3) pressure transmitters
- **Fast response**: 2-5 seconds closure time
- **SIL-2 rated**: Independent of ESD system
- **Single action**: Close valve only (no complex sequence)

**Why HIPPS?**
- **Prevents PSV activation**: Avoids flaring (environmental, economic)
- **Protects equipment**: Prevents mechanical overpressure
- **Independent layer**: Works even if ESD fails

**Voting Logic (2oo3):**
```
PT1=HIGH, PT2=HIGH, PT3=NORMAL → ACTIVATE (2/3 agree)
PT1=HIGH, PT2=NORMAL, PT3=NORMAL → NO ACTION (1/3 only)
```

This prevents spurious trips from single transmitter failure while ensuring safety.

**NeqSim Implementation:**
```java
// HIPPS valve has built-in 2oo3 voting
HIPPSValve hippsValve = new HIPPSValve("HIPPS-001", stream);
hippsValve.addPressureTransmitter(pt1);
hippsValve.addPressureTransmitter(pt2);
hippsValve.addPressureTransmitter(pt3);
hippsValve.setSILRating(2);
```

## 1.5 Theory: Startup Logic with Permissives

**Purpose:** Ensure safe conditions exist before starting equipment.

**Key Concepts:**
- **Permissives**: Conditions that MUST be satisfied before startup
- **Interlocks**: Prevent unsafe startup sequences
- **Staged startup**: Gradual increase to operating conditions

**Common Startup Permissives:**
1. **Pressure**: System must be depressurized
2. **Temperature**: Equipment must be cool
3. **Valve positions**: Isolation valves closed, blowdown closed
4. **Time delay**: Minimum wait after shutdown
5. **Manual approval**: Operator ready

**NeqSim Implementation:**
```java
StartupLogic startupLogic = new StartupLogic("System Startup");

// Define permissives (ALL must be true)
startupLogic.addPermissive(new PressureCondition(separator, 5.0, "<"));
startupLogic.addPermissive(new ValvePositionCondition(bdValve, "<", 5.0));
startupLogic.addPermissive(new TemperatureCondition(separator, 50.0, "<"));
startupLogic.addPermissive(new TimerCondition(10.0));

// Define startup sequence
startupLogic.addAction(new OpenValveAction(inletValve), 0.0);
startupLogic.addAction(new SetValveOpeningAction(inletValve, 50.0), 5.0);
startupLogic.addAction(new SetValveOpeningAction(inletValve, 80.0), 10.0);
```

## 1.6 Theory: Alarm Philosophy

**Alarm Levels:**

| Level | Meaning | Action | Priority |
|-------|---------|--------|----------|
| **LO/HI** | Outside normal range | Operator notification | Low |
| **LOLO/HIHI** | Approaching limits | Automatic action + notification | High |
| **Emergency** | Safety limit exceeded | ESD/HIPPS activation | Critical |

**ISA 18.2 Standard:**
- **HI**: First level warning (yellow)
- **HIHI**: Serious deviation requiring immediate action (orange/red)
- **Response time**: Operator has 10 minutes for HI, immediate for HIHI

**Alarm Actions:**
```
HI Alarm (53 bara):
  → Log event
  → Notify operator
  → NO automatic action
  
HIHI Alarm (56 bara):
  → Log event
  → Notify operator
  → AUTOMATIC: Throttle inlet valve to 60%
  
HIPPS Trigger (59 bara):
  → AUTOMATIC: Close HIPPS valve (3 seconds)
  → INDEPENDENT of alarm system (SIL-2)
```

---

# Part 2: NeqSim Implementation

## 2.1 Build Process System

We'll create a high-pressure gas separation system with:
- **Feed stream**: Mixed gas at 55 bara
- **Inlet control valve**: Flow control (Cv=600)
- **HIPPS valve**: SIL-2 protection (Cv=400, 3s closure)
- **ESD inlet isolation**: Emergency shutdown (Cv=350, 4s closure)
- **HP Separator**: 2m diameter vessel
- **Gas splitter**: Routes to process/PSV/blowdown
- **PSV**: Set at 65 bara (mechanical relief)
- **Blowdown valve**: Fail-safe open for depressurization
- **Flare system**: Emergency relief

### Thermodynamic Setup:
- **Equation of State**: SRK (Soave-Redlich-Kwong)
- **Components**: N₂, CH₄, C₂H₆, C₃H₈, n-C₄H₁₀
- **Mixing Rule**: Classic (type 2)

In [ ]:
// Create process system
ProcessSystem system = new ProcessSystem();

// Define thermodynamic system (SRK equation of state)
SystemInterface feedGas = new SystemSrkEos(298.15, 55.0);  // 25°C, 55 bara
feedGas.addComponent("nitrogen", 1.0);
feedGas.addComponent("methane", 85.0);
feedGas.addComponent("ethane", 10.0);
feedGas.addComponent("propane", 3.0);
feedGas.addComponent("n-butane", 1.0);
feedGas.setMixingRule(2);  // Classic mixing rule

// Create high-pressure feed stream
Stream highPressureFeed = new Stream("HP Feed", feedGas);
highPressureFeed.setFlowRate(15000.0, "kg/hr");
highPressureFeed.setPressure(55.0, "bara");
highPressureFeed.setTemperature(25.0, "C");

// Inlet Control Valve (Flow Control)
ControlValve inletValve = new ControlValve("Inlet Control Valve", highPressureFeed);
inletValve.setPercentValveOpening(80.0);  // 80% open
inletValve.setCv(600.0);  // Valve coefficient

// HIPPS Valve (downstream of inlet control)
HIPPSValve hippsValve = new HIPPSValve("HIPPS-001", inletValve.getOutletStream());
hippsValve.setPercentValveOpening(100.0);  // Normally open
hippsValve.setCv(400.0);
hippsValve.setClosureTime(3.0);  // Fast closure: 3 seconds
hippsValve.setSILRating(2);  // SIL-2 rated system

// ESD Inlet Isolation Valve (downstream of HIPPS)
ESDValve esdInletValve = new ESDValve("ESD Inlet Isolation", hippsValve.getOutletStream());
esdInletValve.setPercentValveOpening(100.0);  // Normally open
esdInletValve.setCv(350.0);
esdInletValve.setStrokeTime(4.0);  // 4 seconds to close
esdInletValve.energize();  // Start energized (open)

// High-pressure separator
Separator separator = new Separator("HP Separator", esdInletValve.getOutletStream());
separator.setCalculateSteadyState(true);
separator.setInternalDiameter(2.0);  // 2 meter diameter

// Gas routing splitter: [process, PSV, blowdown]
Splitter gasSplitter = new Splitter("Gas Splitter", separator.getGasOutStream(), 3);
gasSplitter.setSplitFactors(new double[] {1.0, 0.0, 0.0});  // Normal: all to process

// Process outlet stream
Stream processStream = new Stream("Process Stream", gasSplitter.getSplitStream(0));

// Pressure Safety Valve (PSV)
SafetyValve psv = new SafetyValve("PSV-001", gasSplitter.getSplitStream(1));
psv.setPressureSpec(65.0);  // Set pressure
psv.setFullOpenPressure(67.0);  // Fully open at +2 bara
psv.setBlowdown(7.0);  // Reclose at 58 bara

// ESD Blowdown valve (fail-safe OPEN)
ESDValve bdValve = new ESDValve("ESD Blowdown Valve", gasSplitter.getSplitStream(2));
bdValve.setStrokeTime(5.0);
bdValve.setCv(250.0);
bdValve.setFailSafePosition(100.0);  // Fail-safe to OPEN for blowdown
bdValve.deEnergize();  // Start de-energized (closed)

// Flare header (combines blowdown and PSV relief)
Mixer flareHeader = new Mixer("Flare Header");
flareHeader.addStream(bdValve.getOutletStream());
flareHeader.addStream(psv.getOutletStream());

// Emergency flare
Flare flare = new Flare("Emergency Flare", flareHeader.getOutletStream());
flare.setFlameHeight(50.0);
flare.setRadiantFraction(0.25);

// Add all equipment to process system
system.add(highPressureFeed);
system.add(inletValve);
system.add(hippsValve);
system.add(esdInletValve);
system.add(separator);
system.add(gasSplitter);
system.add(processStream);
system.add(psv);
system.add(bdValve);
system.add(flareHeader);
system.add(flare);

System.out.println("✓ Process system built with " + system.getUnitOperations().size() + " units");
System.out.println("\nEquipment List:");
for (int i = 0; i < system.getUnitOperations().size(); i++) {
    System.out.println("  " + (i+1) + ". " + system.getUnitOperations().get(i).getName());
}

## 2.2 Setup Instrumentation

### Theory: Instrumentation Architecture

**Pressure Monitoring:**
- **PT-101**: Separator pressure (process monitoring)
- **PT-HIPPS-1/2/3**: Three independent transmitters for HIPPS (2oo3 voting)

**Why 2oo3 Voting?**
- **Safety**: 2 transmitters must agree before action
- **Availability**: Single transmitter failure doesn't cause spurious trip
- **SIL-2 Compliance**: Meets safety integrity requirements

**Temperature Monitoring:**
- **TT-101**: Separator temperature

**Manual Activation:**
- **ESD-PB-001**: Emergency shutdown push button

In [ ]:
// Pressure and temperature monitoring on separator
PressureTransmitter separatorPT = new PressureTransmitter("PT-101", separator.getGasOutStream());
TemperatureTransmitter separatorTT = new TemperatureTransmitter("TT-101", separator.getGasOutStream());

// Configure HIPPS pressure transmitters for SIL-2 (2oo3 voting)
// Monitor pressure at inlet control valve outlet (upstream of HIPPS)
PressureTransmitter hippsPT1 = new PressureTransmitter("PT-HIPPS-1", inletValve.getOutletStream());
PressureTransmitter hippsPT2 = new PressureTransmitter("PT-HIPPS-2", inletValve.getOutletStream());
PressureTransmitter hippsPT3 = new PressureTransmitter("PT-HIPPS-3", inletValve.getOutletStream());

// Add pressure transmitters to HIPPS valve (enables 2oo3 voting)
hippsValve.addPressureTransmitter(hippsPT1);
hippsValve.addPressureTransmitter(hippsPT2);
hippsValve.addPressureTransmitter(hippsPT3);

// ESD manual push button
PushButton esdButton = new PushButton("ESD-PB-001");

System.out.println("✓ Instrumentation setup completed:");
System.out.println("  - Separator monitoring: PT-101, TT-101");
System.out.println("  - HIPPS monitoring: PT-HIPPS-1/2/3 (2oo3 voting)");
System.out.println("  - Manual ESD button: ESD-PB-001");

## 2.3 Configure Process Logic

### Implementation Details:

**HIPPS Logic:**
- Single action: Close HIPPS valve immediately
- Independent of ESD system
- Activated by pressure transmitter voting

**ESD Logic (5-step sequence):**
1. Close inlet control valve (t=0s)
2. Close ESD inlet isolation (t=0s)
3. Route gas to blowdown (t=0.5s)
4. Open blowdown valve (t=0.5s)
5. Switch separator to transient mode (t=1.0s)

**Startup Logic:**
- 4 permissives must be satisfied
- 4-step gradual startup sequence
- Valve opens in stages: 0% → 50% → 80%

In [ ]:
// ========================================
// HIPPS Logic (SIL-2) - Fast pressure protection
// ========================================
ESDLogic hippsLogic = new ESDLogic("HIPPS Protection");
hippsLogic.addAction(new CloseValveAction(hippsValve), 0.0);  // Immediate closure

// ========================================
// ESD Logic (SIL-2) - Emergency shutdown
// ========================================
ESDLogic esdLogic = new ESDLogic("ESD Level 1");

// Step 1 & 2: Close inlet valves immediately (t=0s)
esdLogic.addAction(new CloseValveAction(inletValve), 0.0);
esdLogic.addAction(new CloseValveAction(esdInletValve), 0.0);

// Step 3: Route gas to blowdown system (t=0.5s)
esdLogic.addAction(new SetSplitterAction(gasSplitter, new double[]{0.0, 0.0, 1.0}), 0.5);

// Step 4: Open blowdown valve (t=0.5s)
esdLogic.addAction(new EnergizeESDValveAction(bdValve, 100.0), 0.5);

// Step 5: Switch to transient calculation (t=1.0s)
esdLogic.addAction(new SetSeparatorModeAction(separator, false), 1.0);

// Link ESD button to logic
esdButton.linkToLogic(esdLogic);

// ========================================
// Startup Logic with Permissives
// ========================================
StartupLogic startupLogic = new StartupLogic("System Startup");

// Define permissives (ALL must be satisfied before startup)
startupLogic.addPermissive(new PressureCondition(separator, 5.0, "<"));      // Depressurized
startupLogic.addPermissive(new ValvePositionCondition(bdValve, "<", 5.0));   // BD closed
startupLogic.addPermissive(new TemperatureCondition(separator, 50.0, "<"));  // Cool
startupLogic.addPermissive(new TimerCondition(10.0));                        // 10s wait

// Define startup sequence
startupLogic.addAction(new OpenValveAction(inletValve), 0.0);                    // Open
startupLogic.addAction(new SetValveOpeningAction(inletValve, 50.0), 5.0);        // 50% after 5s
startupLogic.addAction(new SetValveOpeningAction(inletValve, 80.0), 10.0);       // 80% after 10s
startupLogic.addAction(new SetSeparatorModeAction(separator, true), 15.0);       // Steady state

System.out.println("✓ Process logic configured:");
System.out.println("  - HIPPS Logic: Single fast-acting protection");
System.out.println("  - ESD Logic: 5-step shutdown sequence");
System.out.println("  - Startup Logic: 4 permissives + 4-step sequence");

## 2.4 Configure Alarm System

### Theory: Alarm Configuration in NeqSim

**AlarmConfig Builder Pattern:**
```java
AlarmConfig config = AlarmConfig.builder("Alarm Name", transmitter)
    .setLOLO(20.0)  // Very low alarm
    .setLO(30.0)    // Low alarm
    .setHI(70.0)    // High alarm
    .setHIHI(85.0)  // Very high alarm
    .build();
```

**ProcessAlarmManager:**
- Centralized alarm coordination
- Evaluates all alarms each timestep
- Triggers registered actions
- Maintains alarm state history

**AlarmActionHandler:**
- Functional interface for alarm-triggered actions
- Executes when alarm activates
- Can trigger logic, adjust valves, etc.

In [ ]:
// Create centralized alarm manager
ProcessAlarmManager alarmManager = new ProcessAlarmManager();

// ========================================
// Pressure Alarms (5 layers)
// ========================================
AlarmConfig pressureAlarm = AlarmConfig.builder("Separator Pressure", separatorPT)
    .setHI(53.0)     // Layer 1: Operator notification
    .setHIHI(56.0)   // Layer 2: Auto throttle inlet valve
    .build();

// HIHI alarm action: Throttle inlet valve to 60%
AlarmActionHandler throttleAction = (alarm, manager) -> {
    System.out.println("  🔧 HIHI Action: Throttling inlet valve to 60%");
    inletValve.setPercentValveOpening(60.0);
};

// Register alarm with actions
alarmManager.addAlarm(pressureAlarm);
alarmManager.registerAlarmAction(pressureAlarm, "HIHI", throttleAction);

// ========================================
// HIPPS Trigger Alarm (Layer 3 - SIL-2)
// ========================================
AlarmConfig hippsAlarm = AlarmConfig.builder("HIPPS Trigger", separatorPT)
    .setHIHI(59.0)   // HIPPS activation pressure
    .build();

AlarmActionHandler hippsAction = (alarm, manager) -> {
    System.out.println("  ⚡ HIPPS TRIGGERED: Fast valve closure (3 seconds)");
    hippsLogic.activate();
};

alarmManager.addAlarm(hippsAlarm);
alarmManager.registerAlarmAction(hippsAlarm, "HIHI", hippsAction);

// ========================================
// ESD Trigger Alarm (Layer 4 - SIL-2)
// ========================================
AlarmConfig esdAlarm = AlarmConfig.builder("ESD Trigger", separatorPT)
    .setHIHI(60.0)   // ESD activation pressure
    .build();

AlarmActionHandler esdAction = (alarm, manager) -> {
    System.out.println("  🚨 ESD TRIGGERED: Emergency shutdown sequence initiated");
    esdLogic.activate();
};

alarmManager.addAlarm(esdAlarm);
alarmManager.registerAlarmAction(esdAlarm, "HIHI", esdAction);

// ========================================
// Temperature Alarms
// ========================================
AlarmConfig tempAlarm = AlarmConfig.builder("Separator Temperature", separatorTT)
    .setHI(40.0)
    .setHIHI(55.0)
    .build();

alarmManager.addAlarm(tempAlarm);

System.out.println("✓ Alarm system configured:");
System.out.println("  - Pressure alarms: HI(53), HIHI(56), HIPPS(59), ESD(60)");
System.out.println("  - Temperature alarms: HI(40), HIHI(55)");
System.out.println("  - Actions registered for HIHI/HIPPS/ESD triggers");

## 2.5 Initialize and Run Steady-State

### Theory: Steady-State vs Transient

**Steady-State Calculation:**
- Solves mass & energy balances assuming constant conditions
- Used for initial plant conditions
- Fast convergence (iterative solver)
- **NeqSim**: `system.run()`

**Transient Calculation:**
- Time-stepping solver
- Simulates dynamic behavior
- Tracks pressure/temperature/composition changes over time
- **NeqSim**: `system.runTransient(timeStep, uuid)`

**Workflow:**
1. Initialize with steady-state
2. Apply disturbances/activate logic
3. Run transient simulation to observe dynamics

In [ ]:
// Create scenario runner and initialize
ProcessScenarioRunner runner = new ProcessScenarioRunner(system);

// Add logic sequences to runner
runner.addLogic(hippsLogic);
runner.addLogic(esdLogic);
runner.addLogic(startupLogic);

// Initialize steady-state solution
System.out.println("\n" + "=".repeat(70));
System.out.println("INITIALIZING STEADY-STATE SOLUTION");
System.out.println("=".repeat(70));

runner.initializeSteadyState();

System.out.println("\n✓ System ready for scenario execution");

---

# Part 3: Scenario Execution & Results

## 3.1 Theory: Scenario Testing Methodology

**Purpose of Scenario Testing:**
- Verify safety systems respond correctly to abnormal conditions
- Validate protection layer independence
- Demonstrate compliance with safety requirements
- Identify potential hazards

**Test Scenarios:**
1. **Normal Startup**: Verify permissives and staged startup
2. **Manual ESD**: Test operator intervention
3. **High Pressure**: Test automatic pressure protection sequence
4. **Equipment Failure**: Test response to utility loss

**ProcessSafetyScenario Builder:**
- **Perturbations**: Change equipment conditions (pressure, flow, etc.)
- **Utility Loss**: Simulate equipment failures
- **Custom Manipulators**: Lambda functions for complex changes

## 3.2 Batch Execution API

The `ScenarioTestRunner.batch()` API enables fluent scenario definition:

```java
testRunner.batch()
    .add("Name", scenario, "Logic", duration, timestep)
    .addDelayed("Name", scenario, "Logic", delay_ms, "Message", duration, timestep)
    .execute();
```

**Benefits:**
- Automatic header & dashboard display
- Consistent scenario numbering
- KPI collection & comparison
- Clean, readable test code

In [ ]:
// Create test runner with automatic KPI collection
ScenarioTestRunner testRunner = new ScenarioTestRunner(runner);

// Execute all scenarios using batch API
testRunner.batch()

    // Scenario 1: Normal startup sequence
    .add("Normal Startup",
         ProcessSafetyScenario.builder("Normal Startup").build(),
         "System Startup",  // Activate startup logic
         30.0,  // Duration: 30 seconds
         1.0)   // Timestep: 1 second

    // Scenario 2: Manual ESD activation (operator pushes button after 5 seconds)
    .addDelayed("Manual ESD",
                ProcessSafetyScenario.builder("Manual ESD").build(),
                "ESD Level 1",  // Activate ESD logic
                5000,  // Delay: 5 seconds
                "⚠️  OPERATOR ACTIVATES ESD BUTTON",
                25.0,  // Duration
                0.5)   // Timestep

    // Scenario 3: High pressure upset (feed pressure increases to 70 bara)
    .addDelayed("High Pressure",
                ProcessSafetyScenario.builder("High Pressure")
                    .customManipulator("HP Feed", equipment -> {
                        if (equipment instanceof Stream) {
                            ((Stream) equipment).setPressure(70.0, "bara");
                            System.out.println("  📈 Feed pressure increased to 70 bara");
                        }
                    }).build(),
                "ESD Level 1",
                8000,  // ESD triggers after 8 seconds
                "🚨 HIGH PRESSURE DETECTED - AUTO ESD TRIGGERED",
                30.0,
                1.0)

    // Scenario 4: Equipment failure (separator utility loss)
    .add("Equip Failure",
         ProcessSafetyScenario.builder("Equipment Failure")
             .utilityLoss("HP Separator").build(),
         null,  // No logic activation
         20.0,
         1.0)

    .execute();  // Run all scenarios and display dashboard

## 3.3 Interpreting Results

### Expected Behavior by Scenario:

**Scenario 1: Normal Startup**
- ✓ All permissives satisfied (P<5 bara, T<50°C, BD closed, 10s timer)
- ✓ Valve opens gradually: 0% → 50% → 80%
- ✓ Separator mode switches to steady-state
- ✓ System reaches normal operating conditions

**Scenario 2: Manual ESD**
- ✓ Operator pushes button at t=5s
- ✓ ESD sequence executes:
  - t=0: Close inlet valves
  - t=0.5s: Route to blowdown
  - t=0.5s: Open blowdown valve
  - t=1.0s: Switch to transient mode
- ✓ System depressurizes to flare

**Scenario 3: High Pressure**
- ✓ Feed pressure jumps to 70 bara (simulating upstream upset)
- ✓ Separator pressure rises
- ✓ Alarms activate in sequence:
  - HI alarm (53 bara) → Operator notification
  - HIHI alarm (56 bara) → Auto throttle to 60%
  - HIPPS trigger (59 bara) → Fast close HIPPS valve
  - ESD trigger (60 bara) → Full shutdown sequence
- ✓ System safely shuts down before PSV opens (65 bara)

**Scenario 4: Equipment Failure**
- ✓ Separator utility loss detected
- ✓ System behavior depends on process configuration
- ✓ May trigger alarms if conditions deteriorate

### KPI Dashboard

The dashboard automatically compares scenarios across:
- **Duration**: Total simulation time
- **Logic Executed**: Which safety sequences ran
- **Errors**: Any simulation errors
- **Status**: Success/failure of scenario

---

# Part 4: Advanced Topics

## 4.1 Creating Custom Logic Classes

You can extend the framework by implementing the `ProcessLogic` interface:

```java
public class CustomLogic implements ProcessLogic {
    private LogicState state = LogicState.IDLE;
    private final String name;
    
    @Override
    public String getName() { return name; }
    
    @Override
    public LogicState getState() { return state; }
    
    @Override
    public void activate() {
        state = LogicState.RUNNING;
        // Custom activation code
    }
    
    @Override
    public void execute(double timeStep) {
        if (state == LogicState.RUNNING) {
            // Time-based logic execution
        }
    }
    
    // Implement remaining methods: deactivate(), reset(),
    // isActive(), isComplete(), getTargetEquipment(),
    // getStatusDescription()
}
```

## 4.2 Transient Blowdown Simulation

For ESD scenarios, switch to transient mode to simulate depressurization:

```java
// In ESD sequence
esdLogic.addAction(new SetSeparatorModeAction(separator, false), 1.0);

// Then run transient calculation
while (time < duration) {
    system.runTransient(timeStep, UUID.randomUUID());
    // Monitor pressure drop, flow rates, etc.
}
```

## 4.3 Integration with Real-Time Systems

**Potential Applications:**
- **Training simulators**: Operator training for emergency procedures
- **Digital twins**: Real-time process monitoring
- **Safety system testing**: Verify SIS logic before deployment
- **Hazard analysis**: HAZOP/LOPA studies

**NeqSim Advantages:**
- **Thermodynamically rigorous**: Accurate phase behavior
- **Equipment models**: Valves, separators, PSVs, etc.
- **Logic framework**: Complete safety system representation
- **Fast execution**: Suitable for real-time applications

---

# Summary & Key Takeaways

## Theoretical Concepts Covered

✅ **Layers of Protection Analysis (LOPA)** - Multiple independent safety barriers  
✅ **Safety Integrity Levels (SIL)** - Risk reduction factors for safety systems  
✅ **Alarm Philosophy** - HI/HIHI alarm hierarchies and response requirements  
✅ **ESD Logic** - Time-sequenced emergency shutdown procedures  
✅ **HIPPS** - Fast-acting pressure protection with voting logic  
✅ **Startup Permissives** - Safe startup interlocks  
✅ **Transient Simulation** - Dynamic process behavior modeling  

## NeqSim Implementation Features

✅ **Equipment Models** - Valves, separators, PSVs, flares  
✅ **Instrumentation** - Pressure/temperature/level transmitters, push buttons  
✅ **Logic Framework** - ESD, HIPPS, Startup, custom logic  
✅ **Alarm System** - Multi-level alarms with automatic actions  
✅ **Scenario Testing** - Batch execution with KPI dashboards  
✅ **Thermodynamics** - Rigorous phase equilibrium (SRK EOS)  
✅ **Steady-State & Transient** - Both calculation modes supported  

## Practical Applications

✅ **Safety Studies** - HAZOP, LOPA, SIL verification  
✅ **Training Simulators** - Operator training for abnormal situations  
✅ **Design Validation** - Verify safety system design before installation  
✅ **Digital Twins** - Real-time process monitoring and optimization  
✅ **Compliance** - Document safety system performance for regulations  

## Next Steps

1. **Modify scenarios**: Add your own process upsets
2. **Custom logic**: Create domain-specific logic classes
3. **Alarm tuning**: Adjust setpoints for your process
4. **Integration**: Connect to real-time data sources
5. **Visualization**: Add plotting of pressure/temperature trends

---

**Documentation:** [NeqSim GitHub](https://github.com/equinor/neqsim)  
**Support:** Report issues on GitHub  
**License:** Apache 2.0

## Appendix: Notebook Structure

This notebook contains **18 cells** organized as follows:

### Part 1: Theory (Cells 1-7)
- Introduction & safety architecture overview
- Layers of Protection Analysis (LOPA)
- ESD Logic theory
- HIPPS theory with 2oo3 voting
- Startup Logic with permissives
- Alarm philosophy & ISA 18.2 standard

### Part 2: Implementation (Cells 8-12)
- Import libraries
- Build process system (11 units)
- Setup instrumentation (2oo3 HIPPS transmitters)
- Configure process logic (HIPPS, ESD, Startup)
- Configure alarm system with automatic actions
- Initialize steady-state

### Part 3: Scenarios (Cells 13-14)
- Batch execution of 4 test scenarios
- Results interpretation & KPI dashboard

### Part 4: Advanced Topics (Cells 15-16)
- Creating custom logic classes
- Transient blowdown simulation
- Real-time integration

### Part 5: Summary (Cells 17-18)
- Key takeaways
- Practical applications
- Next steps

**Total Lines of Code:** ~200 lines  
**Execution Time:** ~2-3 minutes (depends on scenario complexity)  
**Memory Usage:** ~500 MB (typical for NeqSim thermodynamic calculations)